# Create station metadata from `ghcnd-stations.txt` and `ghcnd-countries.txt`

In [ ]:
import urllib.request
import numpy as np
import pandas as pd

URL_station = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt"

with urllib.request.urlopen(URL_station) as file:

    lines = [line.decode('utf-8').rstrip() for line in file]
 
    ID_list = [line[:11] for line in lines]
 
    lat_list = [line[12:20].lstrip() for line in lines]
 
    lon_list = [line[21:30].lstrip() for line in lines]
   
    state_list = [line[38:40].lstrip() for line in lines]
 
    station_name_list = [line[41:71].strip() for line in lines]


metadata__station = pd.DataFrame({  
    'ID' : ID_list,
    'Latitude' : lat_list,
    'Longitude' : lon_list,
    'State' : state_list,
    'StationName' : station_name_list
})

metadata__station['State'] = metadata__station['State'].replace('', np.nan)
metadata__station['Country_code'] = metadata__station.ID.str[:2]

metadata__station

In [ ]:
URL_countries = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-countries.txt"

with urllib.request.urlopen(URL_countries) as file:
    
    lines = [line.decode('utf-8').rstrip().split(" ", 1) for line in file]
    
    Country_code = [line[0] for line in lines]
    
    Country = [line[1] for line in lines]
    
metadata_countries = pd.DataFrame({  
    'Country_code' : Country_code,
    'Country' : Country
})

metadata_countries

In [ ]:
final_metedata = pd.merge(metadata__station, metadata_countries, on = 'Country_code')
#final_metedata.to_csv('station_metadata.csv', index = False)